<LARGE>Select the Runtime > "Change runtime type" menu to enable a GPU accelerator</LARGE>

In [ ]:
!git clone https://github.com/naufalhisyam/rocket-lander.git

Run the cell below if you want to upload saved checkpoint and continue training from that checkpoint

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
#!unzip /content/ddpg_checkpoint.zip 'content/*' -d "/"

Installing required libraries

In [ ]:
!apt install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb ffmpeg xorg-dev python-opengl libboost-all-dev libsdl2-dev swig > /dev/null 2>&1

In [ ]:
!pip install cvxpy
!pip install box2d-py
!pip uninstall stable-baselines -y
!pip uninstall dopamine-rl -y
!pip uninstall pyglet -y
!pip uninstall gym -y
!pip install tensorflow==1.15
!pip install pyglet==1.3.2
!pip install gym==0.9.4
!pip install pyvirtualdisplay

<LARGE>Restart the Runtime first before proceeding below</LARGE>

In [ ]:
%cd /content/rocket-lander
!ls

In [ ]:
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1000, 800))
display.start()

In [ ]:
%tensorflow_version 1.x
from numpy.core.numeric import False_
import tensorflow as tf
from control_and_ai.DDPG.ddpg import DDPG
from control_and_ai.DDPG.train import set_up
#from control_and_ai.DDPG.train_third_model_normalized import train as train_third_model_normalized
#from control_and_ai.DDPG.train_second_model import train as train_second_model
#from control_and_ai.DDPG.train_third_model_unnormalized import train as train_third_model_unnormalized
from control_and_ai.DDPG.train import train as train_first_model

from constants import DEGTORAD
from control_and_ai.DDPG.exploration import OUPolicy
from main_simulation import RocketLander

FLAGS = set_up()

action_bounds = [1, 1, 15*DEGTORAD]

eps = []
eps.append(OUPolicy(0, 0.2, 0.4))
eps.append(OUPolicy(0, 0.2, 0.4))
eps.append(OUPolicy(0, 0.2, 0.4))

simulation_settings = {'Side Engines': True,
                       'Clouds': True,
                       'Vectorized Nozzle': True,
                       'Graph': False,
                       'Render': False,
                       'Starting Y-Pos Constant': 1,
                       'Initial Force': 'random',
                       'Rows': 1,
                       'Columns': 2,
                       'Episodes': 500}
env = RocketLander(simulation_settings)

#Set both line below to False if you want to contniue training from a saved checkpoint
FLAGS.retrain = True #Restore weights if False
FLAGS.test = False #Test the model

FLAGS.num_episodes = 2000
FLAGS.save_state_act = False #Export State and Action of a single episode as .xlsx (only work if num_episodes = 1)
FLAGS.save_reward = False #Export reward log as .xlsx
FLAGS.render_episodes = 1000 #When to render

model_dir = '/content/rocket-lander/saved_model/DDPG/model'

agent = DDPG(
    action_bounds,
    eps,
    env.observation_space.shape[0], #for first model
    actor_learning_rate=0.0001,
    critic_learning_rate=0.001,
    retrain=FLAGS.retrain,
    log_dir=FLAGS.log_dir,
    model_dir=model_dir)
    #batch_size=1000,
    #gamma=0.999)

train_first_model(env, agent, FLAGS)
#train_second_model(env, agent, FLAGS)
#train_third_model_normalized(env, agent, FLAGS)
#train_third_model_unnormalized(env, agent, FLAGS)


Download saved checkpoint

In [ ]:
!zip -r /content/zipped/ddpg_checkpoint.zip /content/rocket-lander/saved_model/DDPG/model

from google.colab import files
files.download("/content/zipped/ddpg_checkpoint.zip")

#If the download doesn't start automatically,
#you can download the file manually from "Files" menu on the left side of the screen